# **Importing necessary Libraries**

---



---



In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

# **Importing MNIST/Fashion_MNIST dataset**

---



---



In [3]:
(X_train,Y_train),(X_test,Y_test)=tf.keras.datasets.mnist.load_data()
(fX_train,fY_train),(fX_test,fY_test)=tf.keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


# **Normalizing Pixel Values between 0-1**

---



---



In [4]:
X_train = np.array(X_train) / 255.0
X_test = np.array(X_test) / 255.0
fX_train = np.array(fX_train) / 255.0
fX_test = np.array(fX_test) / 255.0
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
fX_train = fX_train.reshape(-1, 28, 28, 1)
fX_test = fX_test.reshape(-1, 28, 28, 1)
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
fY_train = to_categorical(fY_train)
fY_test = to_categorical(fY_test)

# **Defining FFN model Using Subclassing method**

---



---



In [5]:
class FFN(tf.keras.models.Model):
  def __init__(self):
    super(FFN,self).__init__()
    self.flatten=tf.keras.layers.Flatten()
    self.dense=tf.keras.layers.Dense(784,activation='relu')
    self.dense2=tf.keras.layers.Dense(512,activation='relu')
    self.dropout=tf.keras.layers.Dropout(rate=0.3)
    self.dense3=tf.keras.layers.Dense(256,activation='relu')
    self.dense4=tf.keras.layers.Dense(10,activation='softmax')
  def call(self,x,training):
    x=self.flatten(x)
    x=self.dense(x)
    x=self.dense2(x)
    x=self.dropout(x,training=training)
    x=self.dense3(x)
    Output=self.dense4(x)
    return Output

# **Initializing models**

---



---



In [6]:
classes=["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]
FMNISTmodel=FFN()
MNISTmodel=FFN()

# **Defining Loss ,Optimizer and Accuracy**

---



---



In [7]:
loss_compute=tf.keras.losses.CategoricalCrossentropy()
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=tf.keras.metrics.CategoricalAccuracy()

# **Manually Backpropogating loss**

---



---



In [8]:
def backpropagation(model,image,Y):
  # image=tf.reshape(image,(-1,784,1))
  with tf.GradientTape() as tape:
    pred=model(image,training=True)
    loss=loss_compute(Y,pred)
  gradient=tape.gradient(loss,model.trainable_variables)
  # optimizer.build(model.trainable_variables)
  optimizer.apply_gradients(zip(gradient,model.trainable_variables))
  return loss,pred

# **Manually training models**

---



---



In [9]:
def train_model(model,X, Y, epochs, batch_size):
    assert tf.shape(X)[0] == tf.shape(Y)[0], "Length of data is not equal to its label"
    A="="
    B=0
    for epoch in range(epochs):
        print("[",end="")
        epoch_loss = 0.0
        epoch_accuracy = 0.0
        for i in range(len(X) // batch_size):
            data = X[i * batch_size:(i + 1) * batch_size]
            label = Y[i * batch_size:(i + 1) * batch_size]
            loss, pred = backpropagation(model,data, label)
            metrics.update_state(label, pred)
            epoch_loss += loss.numpy()
            epoch_accuracy += metrics.result().numpy()
            B+=1
            if B%100==0:
              print(f"{A}",end='')
        epoch_loss /= (len(X) // batch_size)
        epoch_accuracy /= (len(X) // batch_size)
        print(f"]  --  Epoch = {epoch + 1} | Loss = {epoch_loss} | Accuracy = {epoch_accuracy}")

# **Training of model**

---



---



In [ ]:
print("\nTraining model on MNIST:\n")
train_model(MNISTmodel,X_train,Y_train,epochs=10,batch_size=32)

In [10]:
print("\nTraining model on Fasion_MNIST:\n")
train_model(FMNISTmodel,fX_train,fY_train,epochs=10,batch_size=32)


Training model on Fasion_MNIST:

[==================]  --  Epoch = 1 | Loss = 0.4947497535943985 | Accuracy = 0.7708568232218425
[===================]  --  Epoch = 2 | Loss = 0.3772461126208305 | Accuracy = 0.831666999912262
[===================]  --  Epoch = 3 | Loss = 0.33806570211251574 | Accuracy = 0.8469114347139994
[===================]  --  Epoch = 4 | Loss = 0.3136015087465445 | Accuracy = 0.856380134455363
[==================]  --  Epoch = 5 | Loss = 0.2970322691996892 | Accuracy = 0.8633586227416992
[===================]  --  Epoch = 6 | Loss = 0.28276311409870786 | Accuracy = 0.8687650825182597
[===================]  --  Epoch = 7 | Loss = 0.27034747816622257 | Accuracy = 0.8730258831342061
[===================]  --  Epoch = 8 | Loss = 0.2607948694288731 | Accuracy = 0.8765813731829325
[==================]  --  Epoch = 9 | Loss = 0.25129474619925024 | Accuracy = 0.8798009222348531
[===================]  --  Epoch = 10 | Loss = 0.2444482000986735 | Accuracy = 0.8825935522397